# Preprocesamiento de Datos
## Proyecto de Mantenimiento Predictivo - Moto-Compresores

### 🎯 Objetivo del Notebook
Este notebook realiza la **limpieza y preprocesamiento** de los datos operacionales del moto-compresor para prepararlos para el modelado de Machine Learning. El objetivo es consolidar múltiples archivos de sensores, limpiar inconsistencias, manejar valores atípicos y crear un dataset estructurado y confiable.

### 📋 Tareas Principales
1. **Carga y Consolidación**: Integrar datos de 28 archivos Excel de sensores
2. **Limpieza de Estructura**: Estandarizar nombres de columnas y tipos de datos
3. **Conversión Temporal**: Procesar diferentes formatos de fecha/hora
4. **Tratamiento de Valores Faltantes**: Interpolación basada en tiempo
5. **Manejo de Outliers**: Clipping estadístico para preservar información útil
6. **Validación de Calidad**: Métricas de evaluación del procesamiento

### 🛠️ Librerías Utilizadas
- **pandas**: Manipulación y análisis de datos
- **numpy**: Operaciones numéricas
- **pathlib**: Manejo de rutas de archivos
- **warnings**: Supresión de advertencias menores

In [1]:
# Importación de librerías esenciales
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
from datetime import datetime
import re
import gc
import sys
import subprocess

warnings.filterwarnings('ignore')


# Configuración del entorno
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Librerías importadas exitosamente")
print(f"📊 Versión de pandas: {pd.__version__}")
print(f"🔢 Versión de numpy: {np.__version__}")

✅ Librerías importadas exitosamente
📊 Versión de pandas: 2.3.1
🔢 Versión de numpy: 2.3.2


In [2]:
# Configuración de rutas de directorio usando pathlib
# Esto garantiza compatibilidad cross-platform y rutas relativas robustas
# Directorio base del proyecto
base_dir = Path('..')

# Rutas de datos
ruta_raw = base_dir / 'data' / 'raw'
ruta_processed = base_dir / 'data' / 'processed'
ruta_eventos = base_dir / 'eventos'

# Crear directorio processed si no existe
ruta_processed.mkdir(parents=True, exist_ok=True)

# Validación de rutas
print("📁 Configuración de rutas:")
print(f"   Raw data: {ruta_raw} - {'✅ Existe' if ruta_raw.exists() else '❌ No existe'}")
print(f"   Processed: {ruta_processed} - {'✅ Existe' if ruta_processed.exists() else '❌ No existe'}")
print(f"   Eventos: {ruta_eventos} - {'✅ Existe' if ruta_eventos.exists() else '❌ No existe'}")

# Listar archivos disponibles
archivos_excel = list(ruta_raw.glob('*.xls*'))
print(f"\n📊 Archivos de datos encontrados: {len(archivos_excel)} archivos")

📁 Configuración de rutas:
   Raw data: ../data/raw - ✅ Existe
   Processed: ../data/processed - ✅ Existe
   Eventos: ../eventos - ✅ Existe

📊 Archivos de datos encontrados: 28 archivos


## 1. 📁 Función de Carga y Consolidación de Datos

### 🔧 Estrategia Técnica
Desarrollaremos una función robusta que:
- **Detecta automáticamente** la estructura de cada archivo Excel
- **Identifica el encabezado** buscando palabras clave como 'COMPRESOR' y 'MOTOR'
- **Maneja diferentes engines** (.xls con xlrd, .xlsx con openpyxl)
- **Consolida datos** de múltiples archivos manteniendo consistencia

Esta aproximación es porque los archivos industriales pueden tener estructuras variables.

In [3]:
def detectar_fila_encabezado(df_raw):
    """
    Detecta automáticamente la fila donde comienzan los encabezados de columnas
    buscando palabras clave relacionadas con el moto-compresor
    """
    palabras_clave = ['COMPRESOR', 'MOTOR', 'HORA', 'TIEMPO', 'TEMP', 'PRES']
    
    for idx, fila in df_raw.iterrows():
        # Convertir toda la fila a string y buscar palabras clave
        fila_str = ' '.join([str(cell).upper() for cell in fila if pd.notna(cell)])
        
        # Si encontramos al menos 2 palabras clave, probablemente es el encabezado
        coincidencias = sum(1 for palabra in palabras_clave if palabra in fila_str)
        if coincidencias >= 2:
            return idx
    
    return None

def cargar_archivo_sensor(file_path):
    """
    Carga un archivo de sensores con detección automática de estructura
    """
    try:
        # Cargar archivo completo para análisis de estructura
        if file_path.suffix == '.xlsx':
            df_raw = pd.read_excel(file_path, header=None, engine='openpyxl')
        else:
            df_raw = pd.read_excel(file_path, header=None, engine='xlrd')
        
        # Detectar fila de encabezado
        header_row = detectar_fila_encabezado(df_raw)
        
        if header_row is None:
            print(f"⚠️  No se detectó encabezado automáticamente en {file_path.name}, usando fila 0")
            header_row = 0
        
        # Cargar datos con el encabezado correcto
        if file_path.suffix == '.xlsx':
            df = pd.read_excel(file_path, 
                             header=header_row,
                             skiprows=range(0, header_row) if header_row > 0 else None,
                             engine='openpyxl')
        else:
            df = pd.read_excel(file_path, 
                             header=header_row,
                             skiprows=range(0, header_row) if header_row > 0 else None,
                             engine='xlrd')
        
        # Información básica del archivo
        print(f"📋 {file_path.name}: {df.shape[0]} filas, {df.shape[1]} columnas (encabezado en fila {header_row})")
        
        return df, True
        
    except Exception as e:
        print(f"❌ Error cargando {file_path.name}: {str(e)}")
        return None, False

def cargar_y_consolidar_datos(ruta_raw):
    """
    Carga y consolida todos los archivos de sensores de la ruta especificada
    """
    archivos_excel = list(ruta_raw.glob('*.xls*'))
    
    if not archivos_excel:
        raise FileNotFoundError(f"No se encontraron archivos Excel en {ruta_raw}")
    
    print(f"🔄 Iniciando consolidación de {len(archivos_excel)} archivos...\n")
    
    dataframes_lista = []
    archivos_exitosos = []
    archivos_fallidos = []
    
    for archivo in sorted(archivos_excel):  # Ordenar para procesamiento consistente
        df, exito = cargar_archivo_sensor(archivo)
        
        if exito and df is not None and not df.empty:
            dataframes_lista.append(df)
            archivos_exitosos.append(archivo.name)
        else:
            archivos_fallidos.append(archivo.name)
    
    if not dataframes_lista:
        raise ValueError("No se pudo cargar ningún archivo exitosamente")
    
    # Consolidar todos los DataFrames
    print(f"\n🔗 Consolidando {len(dataframes_lista)} DataFrames...")
    df_consolidado = pd.concat(dataframes_lista, ignore_index=True, sort=False)
    
    # Reporte de consolidación
    print(f"\n📊 Resumen de Consolidación:")
    print(f"   ✅ Archivos exitosos: {len(archivos_exitosos)}")
    print(f"   ❌ Archivos fallidos: {len(archivos_fallidos)}")
    print(f"   📈 Dataset final: {df_consolidado.shape[0]:,} filas, {df_consolidado.shape[1]} columnas")
    
    if archivos_fallidos:
        print(f"\n⚠️  Archivos que fallaron: {', '.join(archivos_fallidos)}")
    
    return df_consolidado, archivos_exitosos, archivos_fallidos

# Ejecutar consolidación
print("🚀 Ejecutando carga y consolidación de datos...")
df_raw, exitosos, fallidos = cargar_y_consolidar_datos(ruta_raw)

🚀 Ejecutando carga y consolidación de datos...
🔄 Iniciando consolidación de 28 archivos...

📋 01-2024.xls: 745 filas, 34 columnas (encabezado en fila 2)
📋 01-2025.xls: 745 filas, 34 columnas (encabezado en fila 2)
📋 02-2024..xls: 697 filas, 34 columnas (encabezado en fila 2)
📋 02-2025.xls: 673 filas, 34 columnas (encabezado en fila 2)
📋 03-2024..xls: 745 filas, 34 columnas (encabezado en fila 2)
📋 03-2025.xls: 745 filas, 34 columnas (encabezado en fila 2)
📋 04-2024..xls: 721 filas, 34 columnas (encabezado en fila 2)
📋 04-2025.xls: 721 filas, 34 columnas (encabezado en fila 2)
📋 05-2024..xls: 745 filas, 34 columnas (encabezado en fila 2)
📋 06-2024..xls: 721 filas, 34 columnas (encabezado en fila 2)
📋 07-2024..xls: 745 filas, 34 columnas (encabezado en fila 2)
📋 08-2024.xls: 745 filas, 34 columnas (encabezado en fila 2)
📋 09-2024.xls: 721 filas, 34 columnas (encabezado en fila 2)
📋 1-2023...xls: 745 filas, 34 columnas (encabezado en fila 2)
📋 10-2023...xls: 745 filas, 34 columnas (encabe

## 2. 🧹 Pipeline de Limpieza de Datos

### 📝 Justificación de la Estrategia
Los datos industriales presentan desafíos típicos:
- **Nombres inconsistentes**: Espacios, saltos de línea, caracteres especiales
- **Formatos temporales variables**: Números de Excel vs. strings de fecha
- **Tipos de datos mixtos**: Strings donde deberían ser numéricos

Implementaremos un pipeline robusto que maneja estos problemas sistemáticamente.

In [4]:
def limpiar_nombres_columnas(df):
    """
    Limpia y estandariza los nombres de columnas a formato snake_case
    """
    print("🔤 Limpiando nombres de columnas...")
    
    # Mostrar algunos nombres originales para referencia
    print(f"\n📋 Muestra de nombres originales:")
    for i, col in enumerate(df.columns[:5]):
        print(f"   {i+1}. '{col}'")
    
    nombres_limpios = []
    
    for col in df.columns:
        # Convertir a string si no lo es
        nombre = str(col)
        
        # Convertir a minúsculas
        nombre = nombre.lower()
        
        # Reemplazar vocales con acento por las vocales sin acento
        nombre = nombre.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
        
        # Reemplazar saltos de línea y espacios múltiples por un espacio
        nombre = re.sub(r'\s+', ' ', nombre)
        
        # Reemplazar espacios por guiones bajos
        nombre = nombre.replace(' ', '_')
        
        # Eliminar caracteres especiales, mantener solo letras, números y guiones bajos
        nombre = re.sub(r'[^a-z0-9_]', '', nombre)
        
        # Eliminar guiones bajos múltiples
        nombre = re.sub(r'_+', '_', nombre)
        
        # Eliminar guiones bajos al inicio y final
        nombre = nombre.strip('_')
        
        # Si el nombre está vacío, generar uno genérico
        if not nombre:
            nombre = f'columna_{len(nombres_limpios)}'
        
        
        nombres_limpios.append(nombre)
    
    # Manejar nombres duplicados
    nombres_finales = []
    contador_nombres = {}
    
    for nombre in nombres_limpios:
        if nombre in contador_nombres:
            contador_nombres[nombre] += 1
            nombre_final = f"{nombre}_{contador_nombres[nombre]}"
        else:
            contador_nombres[nombre] = 0
            nombre_final = nombre
        
        nombres_finales.append(nombre_final)
    
    # Aplicar nombres limpios
    df.columns = nombres_finales
    
    print(f"\n📋 Muestra de nombres limpios:")
    for i, col in enumerate(df.columns[:5]):
        print(f"   {i+1}. '{col}'")
    
    print(f"\n✅ Limpieza de nombres completada: {len(nombres_finales)} columnas procesadas")
    
    return df

# Ejecutar limpieza de nombres
df_clean = limpiar_nombres_columnas(df_raw.copy())

🔤 Limpiando nombres de columnas...

📋 Muestra de nombres originales:
   1. 'ESTADO'
   2. 'Hora'
   3. 'RPM'
   4. 'Presión
Succión'
   5. 'Presión
Intermedia'

📋 Muestra de nombres limpios:
   1. 'estado'
   2. 'hora'
   3. 'rpm'
   4. 'presion_succion'
   5. 'presion_intermedia'

✅ Limpieza de nombres completada: 68 columnas procesadas


In [5]:
def detectar_columna_tiempo(df):
    """
    Detecta la columna de tiempo basándose en nombres comunes y contenido
    """
    nombres_tiempo = ['hora', 'tiempo', 'time', 'fecha', 'date', 'timestamp']
    
    # Buscar por nombre
    for col in df.columns:
        if any(nombre in col.lower() for nombre in nombres_tiempo):
            return col
    
    # Si no encontramos por nombre, buscar la primera columna que parezca temporal
    for col in df.columns:
        try:
            # Verificar si los valores pueden ser fechas
            muestra = df[col].dropna().head(100)
            if muestra.empty:
                continue
                
            # Intentar conversión a datetime
            pd.to_datetime(muestra, errors='raise')
            return col
        except:
            continue
    
    return None

def convertir_tiempo_excel_inteligente(serie_tiempo):
    """
    Convierte una serie de tiempo manejando diferentes formatos:
    - Números de Excel (días desde 1899-12-30)
    - Strings de fecha/hora
    - Timestamps ya convertidos
    """
    if serie_tiempo.empty:
        return serie_tiempo
    
    # Tomar una muestra para detectar formato
    muestra = serie_tiempo.dropna().head(10)
    
    if muestra.empty:
        return serie_tiempo
    
    primer_valor = muestra.iloc[0]
    
    try:
        # Caso 1: Ya es datetime
        if pd.api.types.is_datetime64_any_dtype(serie_tiempo):
            print("   ✅ Columna ya en formato datetime")
            return serie_tiempo
        
        # Caso 2: Números de Excel (típicamente entre 40000-50000 para fechas 2009-2037)
        if pd.api.types.is_numeric_dtype(serie_tiempo):
            if isinstance(primer_valor, (int, float)) and 30000 < primer_valor < 60000:
                print(f"   🔢 Detectado formato numérico Excel (ej: {primer_valor})")
                return pd.to_datetime(serie_tiempo, unit='D', origin='1899-12-30', errors='coerce')
        
        # Caso 3: Strings de fecha/hora
        if isinstance(primer_valor, str):
            print(f"   📝 Detectado formato string (ej: '{primer_valor}')")
            return pd.to_datetime(serie_tiempo, errors='coerce', infer_datetime_format=True)
        
        # Caso 4: Intentar conversión general
        print(f"   🔄 Intentando conversión general para tipo: {type(primer_valor)}")
        return pd.to_datetime(serie_tiempo, errors='coerce')
        
    except Exception as e:
        print(f"   ⚠️  Error en conversión temporal: {str(e)}")
        return pd.to_datetime(serie_tiempo, errors='coerce')

def convertir_tipos_datos(df):
    """
    Convierte los tipos de datos apropiadamente:
    - Columna de tiempo a datetime
    - Resto de columnas a numérico (float64)
    """
    print("🔄 Convirtiendo tipos de datos...")
    
    # Detectar columna de tiempo
    col_tiempo = detectar_columna_tiempo(df)
    
    if col_tiempo:
        print(f"⏰ Columna de tiempo detectada: '{col_tiempo}'")
        
        # Convertir columna de tiempo
        df[col_tiempo] = convertir_tiempo_excel_inteligente(df[col_tiempo])
        
        # Verificar conversión exitosa
        valores_validos = df[col_tiempo].notna().sum()
        total_valores = len(df[col_tiempo])
        
        print(f"   📊 Conversión temporal: {valores_validos:,}/{total_valores:,} valores válidos ({valores_validos/total_valores*100:.1f}%)")
        
        if valores_validos > 0:
            print(f"   📅 Rango temporal: {df[col_tiempo].min()} a {df[col_tiempo].max()}")
            
            # Establecer como índice
            df_indexed = df.set_index(col_tiempo)
            print(f"   ✅ Columna '{col_tiempo}' establecida como índice")
        else:
            print(f"   ❌ Conversión temporal falló, manteniendo índice numérico")
            df_indexed = df
    else:
        print("⚠️  No se detectó columna de tiempo, manteniendo índice numérico")
        df_indexed = df
    
    # Convertir columnas restantes a numérico
    print("\n🔢 Convirtiendo columnas numéricas...")
    
    columnas_numericas = [col for col in df_indexed.columns if col != col_tiempo]
    conversiones_exitosas = 0
    
    for col in columnas_numericas:
        try:
            # Intentar conversión a numérico
            valores_originales = df_indexed[col].notna().sum()
            df_indexed[col] = pd.to_numeric(df_indexed[col], errors='coerce')
            valores_finales = df_indexed[col].notna().sum()
            
            if valores_finales >= valores_originales * 0.8:  # 80% de éxito mínimo
                conversiones_exitosas += 1
            else:
                print(f"   ⚠️  {col}: conversión perdió muchos valores ({valores_finales}/{valores_originales})")
                
        except Exception as e:
            print(f"   ❌ Error convirtiendo {col}: {str(e)}")
    
    print(f"   ✅ Conversiones numéricas exitosas: {conversiones_exitosas}/{len(columnas_numericas)} columnas")
    
    return df_indexed

# Ejecutar conversión de tipos
df_typed = convertir_tipos_datos(df_clean)

🔄 Convirtiendo tipos de datos...
⏰ Columna de tiempo detectada: 'hora'
   ✅ Columna ya en formato datetime
   📊 Conversión temporal: 19,752/20,523 valores válidos (96.2%)
   📅 Rango temporal: 2023-01-01 00:00:00 a 2025-04-30 23:00:00
   ✅ Columna 'hora' establecida como índice

🔢 Convirtiendo columnas numéricas...
   ⚠️  estado: conversión perdió muchos valores (0/19752)
   ⚠️  unnamed_0: conversión perdió muchos valores (0/744)
   ✅ Conversiones numéricas exitosas: 65/67 columnas


In [6]:
# Verificar columnas y datos actuales
print(df_typed.head(2))
df_typed.info()

            estado         rpm  presion_succion  presion_intermedia  presion_descarga  pres_aceite_comp  temp_cilindro_1  temp_cilindro_2  temp_cilindro_3  temp_cilindro_4  presion_aceite_motor  presion_agua  presion_mult_adm_izq  presion_mult_adm_der  presion_carter  temp_aceite_motor  temp_agua_motor  temp_mult_adm_izq  temp_mult_adm_der  temp_cil_1_l  temp_cil_1_r  temp_cil_2_l  temp_cil_2_r  temp_cil_3_l  temp_cil_3_r  temp_mult_esc_izq  temp_cil_4_l  temp_cil_4_r  temp_cil_5_l  temp_cil_5_r  temp_cil_6_l  temp_cil_6_r  temp_mult_esc_der  unnamed_0            unnamed_1  unnamed_2  unnamed_3  unnamed_4  unnamed_5  unnamed_6  unnamed_7  unnamed_8  unnamed_9  unnamed_10  unnamed_11  unnamed_12  unnamed_13  unnamed_14  unnamed_15  unnamed_16  unnamed_17  unnamed_18  unnamed_19  unnamed_20  unnamed_21  unnamed_22  unnamed_23  unnamed_24  unnamed_25  unnamed_26  unnamed_27  unnamed_28  unnamed_29  unnamed_30  unnamed_31  unnamed_32  unnamed_33
hora                                        

## 3. 🔧 Tratamiento de Valores Faltantes y Atípicos

### ⚠️ Borrar registro de nulos y columnas no identificadas

### 📊 Estrategia para Valores Faltantes
Para series temporales industriales, la **interpolación basada en tiempo** es superior a otros métodos porque:
- **Preserva tendencias temporales** naturales del proceso industrial
- **Mantiene correlaciones** entre variables del moto-compresor
- **Es más realista** que forward-fill o valores promedio estáticos

### 📈 Estrategia para Outliers
Aplicaremos **clipping estadístico global** (percentiles 1-99) porque:
- **Preserva información** de eventos operacionales reales
- **Elimina errores de medición** extremos sin perder datos útiles
- **Mantiene consistencia** en rangos operacionales para el modelo

In [7]:
# Eliminar filas con todos los valores vacíos o nulos
df_typed.dropna(inplace=True, how='all')

# Eliminar columnas que su nombre empiece con 'unnamed'
unnamed_cols = [col for col in df_typed.columns if col.startswith('unnamed')]
df_typed.drop(columns=unnamed_cols, inplace=True)

df_typed.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20523 entries, NaT to 2023-09-30 23:00:00
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   estado                0 non-null      float64
 1   rpm                   19678 non-null  float64
 2   presion_succion       19679 non-null  float64
 3   presion_intermedia    19679 non-null  float64
 4   presion_descarga      19679 non-null  float64
 5   pres_aceite_comp      19678 non-null  float64
 6   temp_cilindro_1       19678 non-null  float64
 7   temp_cilindro_2       19678 non-null  float64
 8   temp_cilindro_3       19678 non-null  float64
 9   temp_cilindro_4       19678 non-null  float64
 10  presion_aceite_motor  19678 non-null  float64
 11  presion_agua          19678 non-null  float64
 12  presion_mult_adm_izq  19678 non-null  float64
 13  presion_mult_adm_der  19678 non-null  float64
 14  presion_carter        19678 non-null  float64
 15  

In [8]:
def analizar_valores_faltantes(df):
    """
    Analiza el patrón de valores faltantes en el dataset
    """
    print("🔍 Analizando valores faltantes...")
    
    total_filas = len(df)
    
    # Calcular estadísticas de valores faltantes por columna
    missing_stats = pd.DataFrame({
        'columna': df.columns,
        'valores_faltantes': df.isnull().sum(),
        'porcentaje_faltante': (df.isnull().sum() / total_filas * 100).round(2)
    })
    
    # Ordenar por porcentaje de valores faltantes
    missing_stats = missing_stats.sort_values('porcentaje_faltante', ascending=False)
    
    # Mostrar estadísticas generales
    total_missing = df.isnull().sum().sum()
    total_valores = df.size
    porcentaje_total = (total_missing / total_valores * 100)
    
    print(f"\n📊 Resumen General:")
    print(f"   Total de valores: {total_valores:,}")
    print(f"   Valores faltantes: {total_missing:,} ({porcentaje_total:.2f}%)")
    print(f"   Columnas con valores faltantes: {(missing_stats['valores_faltantes'] > 0).sum()}")
    
    # Mostrar columnas con más valores faltantes
    columnas_con_faltantes = missing_stats[missing_stats['valores_faltantes'] > 0]
    
    if not columnas_con_faltantes.empty:
        print(f"\n📋 Top 10 columnas con más valores faltantes:")
        for _, row in columnas_con_faltantes.head(10).iterrows():
            print(f"   {row['columna']}: {row['valores_faltantes']:,} ({row['porcentaje_faltante']}%)")
    else:
        print("\n✅ ¡No hay valores faltantes en el dataset!")
    
    return missing_stats

def tratar_valores_faltantes(df):
    """
    Trata los valores faltantes usando interpolación temporal
    """
    print("\n🔧 Aplicando tratamiento de valores faltantes...")
    
    # Estadísticas antes del tratamiento
    missing_antes = df.isnull().sum().sum()
    
    if missing_antes == 0:
        print("✅ No hay valores faltantes que tratar")
        return df, 0, 0
    
    try:
        # Crear una copia del DataFrame para evitar modificar el original
        df_filled = df.copy()
        
        # Aplicar interpolación temporal si tenemos índice de tiempo
        if isinstance(df.index, pd.DatetimeIndex):
            print("   📅 Aplicando interpolación temporal (method='time')...")
            # Verificar si hay NaNs en el índice que podrían causar problemas
            if df.index.isna().any():
                print("   ⚠️  Advertencia: Índice contiene valores NaN, usando interpolación lineal")
                df_filled = df_filled.interpolate(method='linear', limit_direction='both')
            else:
                df_filled = df_filled.interpolate(method='time', limit_direction='both')
        else:
            print("   📈 Aplicando interpolación lineal (no hay índice temporal)...")
            df_filled = df_filled.interpolate(method='linear', limit_direction='both')
        
        # Para valores que siguen faltando al inicio/final, usar backward/forward fill
        # Reemplazar métodos deprecated
        df_filled = df_filled.bfill().ffill()
        
        # Estadísticas después del tratamiento
        missing_despues = df_filled.isnull().sum().sum()
        valores_interpolados = missing_antes - missing_despues
        
        print(f"   📊 Resultados:")
        print(f"      Valores faltantes antes: {missing_antes:,}")
        print(f"      Valores faltantes después: {missing_despues:,}")
        print(f"      Valores interpolados: {valores_interpolados:,}")
        
        if missing_antes > 0:
            print(f"      Tasa de éxito: {(valores_interpolados/missing_antes*100):.1f}%")
        
        return df_filled, missing_antes, valores_interpolados
        
    except Exception as e:
        print(f"   ❌ Error durante la interpolación: {str(e)}")
        print("   🔄 Intentando método alternativo con forward/backward fill únicamente...")
        
        try:
            # Método alternativo: solo usar forward/backward fill
            df_filled = df.copy()
            df_filled = df_filled.bfill().ffill()
            
            missing_despues = df_filled.isnull().sum().sum()
            valores_interpolados = missing_antes - missing_despues
            
            print(f"   📊 Resultados (método alternativo):")
            print(f"      Valores faltantes antes: {missing_antes:,}")
            print(f"      Valores faltantes después: {missing_despues:,}")
            print(f"      Valores rellenados: {valores_interpolados:,}")
            
            if missing_antes > 0:
                print(f"      Tasa de éxito: {(valores_interpolados/missing_antes*100):.1f}%")
            
            return df_filled, missing_antes, valores_interpolados
            
        except Exception as e2:
            print(f"   ❌ Error también en método alternativo: {str(e2)}")
            print("   ⚠️  Retornando DataFrame original sin cambios")
            return df, missing_antes, 0


# Ejecutar análisis y tratamiento de valores faltantes
missing_stats = analizar_valores_faltantes(df_typed)
df_no_missing, missing_original, interpolated = tratar_valores_faltantes(df_typed)



🔍 Analizando valores faltantes...

📊 Resumen General:
   Total de valores: 677,259
   Valores faltantes: 47,583 (7.03%)
   Columnas con valores faltantes: 33

📋 Top 10 columnas con más valores faltantes:
   estado: 20,523 (100.0%)
   temp_mult_esc_izq: 848 (4.13%)
   temp_cil_5_l: 847 (4.13%)
   temp_cil_6_r: 847 (4.13%)
   temp_cil_6_l: 847 (4.13%)
   temp_mult_esc_der: 847 (4.13%)
   temp_cil_2_r: 847 (4.13%)
   temp_mult_adm_der: 847 (4.13%)
   temp_mult_adm_izq: 847 (4.13%)
   rpm: 845 (4.12%)

🔧 Aplicando tratamiento de valores faltantes...
   📅 Aplicando interpolación temporal (method='time')...
   ⚠️  Advertencia: Índice contiene valores NaN, usando interpolación lineal
   📊 Resultados:
      Valores faltantes antes: 47,583
      Valores faltantes después: 20,523
      Valores interpolados: 27,060
      Tasa de éxito: 56.9%


In [9]:
def analizar_outliers(df):
    """
    Analiza outliers en las columnas numéricas del dataset
    """
    print("🔍 Analizando outliers en columnas numéricas...")
    
    # Seleccionar solo columnas numéricas
    cols_numericas = df.select_dtypes(include=[np.number]).columns
    
    if len(cols_numericas) == 0:
        print("⚠️  No se encontraron columnas numéricas")
        return pd.DataFrame()
    
    outlier_stats = []
    
    for col in cols_numericas:
        serie = df[col].dropna()
        
        if len(serie) == 0:
            continue
        
        # Calcular percentiles
        q01 = serie.quantile(0.01)
        q99 = serie.quantile(0.99)
        q25 = serie.quantile(0.25)
        q75 = serie.quantile(0.75)
        
        # Contar outliers usando método de percentiles
        outliers_bajos = (serie < q01).sum()
        outliers_altos = (serie > q99).sum()
        total_outliers = outliers_bajos + outliers_altos
        
        # Contar outliers usando método IQR (para comparación)
        iqr = q75 - q25
        lower_iqr = q25 - 1.5 * iqr
        upper_iqr = q75 + 1.5 * iqr
        outliers_iqr = ((serie < lower_iqr) | (serie > upper_iqr)).sum()
        
        outlier_stats.append({
            'columna': col,
            'total_valores': len(serie),
            'outliers_percentil': total_outliers,
            'outliers_iqr': outliers_iqr,
            'porcentaje_percentil': (total_outliers / len(serie) * 100),
            'porcentaje_iqr': (outliers_iqr / len(serie) * 100),
            'q01': q01,
            'q99': q99,
            'min_original': serie.min(),
            'max_original': serie.max()
        })
    
    outlier_df = pd.DataFrame(outlier_stats)
    outlier_df = outlier_df.sort_values('porcentaje_percentil', ascending=False)
    
    # Resumen general
    total_outliers_percentil = outlier_df['outliers_percentil'].sum()
    total_valores = outlier_df['total_valores'].sum()
    
    print(f"\n📊 Resumen de Outliers:")
    print(f"   Columnas analizadas: {len(cols_numericas)}")
    print(f"   Total outliers (percentil 1-99): {total_outliers_percentil:,} ({total_outliers_percentil/total_valores*100:.2f}%)")
    
    # Mostrar top columnas con más outliers
    print(f"\n📋 Top 10 columnas con más outliers (método percentil):")
    for _, row in outlier_df.head(10).iterrows():
        print(f"   {row['columna']}: {row['outliers_percentil']:,} outliers ({row['porcentaje_percentil']:.1f}%)")
    
    return outlier_df

def aplicar_clipping_outliers(df):
    """
    Aplica clipping de outliers usando percentiles 1 y 99
    """
    print("\n🔧 Aplicando clipping de outliers...")
    
    # Seleccionar columnas numéricas
    cols_numericas = df.select_dtypes(include=[np.number]).columns
    
    if len(cols_numericas) == 0:
        print("⚠️  No se encontraron columnas numéricas para procesar")
        return df, 0
    
    df_clipped = df.copy()
    total_clipped = 0
    
    print(f"   📊 Procesando {len(cols_numericas)} columnas numéricas...")
    
    for col in cols_numericas:
        serie_original = df_clipped[col]
        serie_valida = serie_original.dropna()
        
        if len(serie_valida) == 0:
            continue
        
        # Calcular límites de clipping
        limite_inferior = serie_valida.quantile(0.01)
        limite_superior = serie_valida.quantile(0.99)
        
        # Aplicar clipping
        serie_clipped = serie_original.clip(lower=limite_inferior, upper=limite_superior)
        
        # Contar valores modificados
        valores_modificados = (serie_original != serie_clipped).sum()
        total_clipped += valores_modificados
        
        # Actualizar la columna
        df_clipped[col] = serie_clipped
        
        if valores_modificados > 0:
            porcentaje = (valores_modificados / len(serie_original) * 100)
            print(f"      {col}: {valores_modificados:,} valores clipped ({porcentaje:.1f}%) [{limite_inferior:.2f}, {limite_superior:.2f}]")
    
    print(f"\n   ✅ Clipping completado: {total_clipped:,} valores modificados en total")
    
    return df_clipped, total_clipped

# Ejecutar análisis y tratamiento de outliers
outlier_stats = analizar_outliers(df_no_missing)
df_final, total_clipped = aplicar_clipping_outliers(df_no_missing)

🔍 Analizando outliers en columnas numéricas...

📊 Resumen de Outliers:
   Columnas analizadas: 33
   Total outliers (percentil 1-99): 11,958 (1.82%)

📋 Top 10 columnas con más outliers (método percentil):
   rpm: 412 outliers (2.0%)
   presion_intermedia: 412 outliers (2.0%)
   presion_descarga: 412 outliers (2.0%)
   pres_aceite_comp: 412 outliers (2.0%)
   presion_agua: 412 outliers (2.0%)
   presion_mult_adm_izq: 412 outliers (2.0%)
   presion_aceite_motor: 412 outliers (2.0%)
   temp_mult_adm_der: 412 outliers (2.0%)
   temp_cil_4_l: 412 outliers (2.0%)
   temp_cil_6_r: 412 outliers (2.0%)

🔧 Aplicando clipping de outliers...
   📊 Procesando 33 columnas numéricas...
      rpm: 412 valores clipped (2.0%) [-106.41, 872.91]
      presion_succion: 411 valores clipped (2.0%) [-74.85, 486.60]
      presion_intermedia: 412 valores clipped (2.0%) [-117.62, 496.31]
      presion_descarga: 412 valores clipped (2.0%) [-124.46, 483.54]
      pres_aceite_comp: 412 valores clipped (2.0%) [-24.55

## 4. 📊 Validación de Calidad de Datos

### 🎯 Métricas de Calidad
Evaluaremos la calidad del preprocesamiento mediante métricas cuantitativas que demuestren la efectividad de cada paso del pipeline de limpieza.

In [10]:
def generar_reporte_calidad(df_original, df_final, missing_original, interpolated, total_clipped):
    """
    Genera un reporte completo de la calidad del preprocesamiento
    """
    print("📋 REPORTE DE CALIDAD DEL PREPROCESAMIENTO")
    print("=" * 60)
    
    # 1. Estadísticas dimensionales
    print("\n1️⃣  DIMENSIONES DEL DATASET")
    print(f"   Filas originales: {len(df_original):,}")
    print(f"   Filas finales: {len(df_final):,}")
    print(f"   Columnas originales: {df_original.shape[1]}")
    print(f"   Columnas finales: {df_final.shape[1]}")
    
    # 2. Calidad de datos
    print("\n2️⃣  CALIDAD DE DATOS")
    missing_final = df_final.isnull().sum().sum()
    total_valores = df_final.size
    
    print(f"   Valores faltantes originales: {missing_original:,}")
    print(f"   Valores interpolados: {interpolated:,}")
    print(f"   Valores faltantes finales: {missing_final:,}")
    print(f"   Tasa de completitud: {((total_valores - missing_final) / total_valores * 100):.2f}%")
    
    # 3. Tratamiento de outliers
    print("\n3️⃣  TRATAMIENTO DE OUTLIERS")
    print(f"   Valores clipped: {total_clipped:,}")
    print(f"   Porcentaje de valores modificados: {(total_clipped / total_valores * 100):.2f}%")
    
    # 4. Tipos de datos
    print("\n4️⃣  TIPOS DE DATOS")
    tipos_finales = df_final.dtypes.value_counts()
    for tipo, cantidad in tipos_finales.items():
        print(f"   {tipo}: {cantidad} columnas")
    
    # 5. Información temporal
    print("\n5️⃣  INFORMACIÓN TEMPORAL")
    if isinstance(df_final.index, pd.DatetimeIndex):
        print(f"   Índice temporal: ✅ Configurado")
        print(f"   Rango temporal: {df_final.index.min()} a {df_final.index.max()}")
        print(f"   Duración total: {df_final.index.max() - df_final.index.min()}")
    else:
        print(f"   Índice temporal: ❌ No configurado (índice numérico)")
    
    # 6. Resumen de columnas
    print("\n6️⃣  RESUMEN DE COLUMNAS")
    cols_numericas = df_final.select_dtypes(include=[np.number]).columns
    cols_temporales = df_final.select_dtypes(include=['datetime']).columns
    cols_otros = df_final.select_dtypes(exclude=[np.number, 'datetime']).columns
    
    print(f"   Columnas numéricas: {len(cols_numericas)}")
    print(f"   Columnas temporales: {len(cols_temporales)}")
    print(f"   Otras columnas: {len(cols_otros)}")
    
    # 7. Estadísticas descriptivas básicas
    if len(cols_numericas) > 0:
        print("\n7️⃣  ESTADÍSTICAS DESCRIPTIVAS (Columnas Numéricas)")
        stats_desc = df_final[cols_numericas].describe()
        print(f"   Promedio de medias: {stats_desc.loc['mean'].mean():.2f}")
        print(f"   Promedio de desv. estándar: {stats_desc.loc['std'].mean():.2f}")
        print(f"   Rango de valores mínimos: [{stats_desc.loc['min'].min():.2f}, {stats_desc.loc['min'].max():.2f}]")
        print(f"   Rango de valores máximos: [{stats_desc.loc['max'].min():.2f}, {stats_desc.loc['max'].max():.2f}]")
    
    print("\n" + "=" * 60)
    print("✅ PREPROCESAMIENTO COMPLETADO EXITOSAMENTE")
    print("=" * 60)

# Generar reporte de calidad
generar_reporte_calidad(df_raw, df_final, missing_original, interpolated, total_clipped)

📋 REPORTE DE CALIDAD DEL PREPROCESAMIENTO

1️⃣  DIMENSIONES DEL DATASET
   Filas originales: 20,523
   Filas finales: 20,523
   Columnas originales: 68
   Columnas finales: 33

2️⃣  CALIDAD DE DATOS
   Valores faltantes originales: 47,583
   Valores interpolados: 27,060
   Valores faltantes finales: 20,523
   Tasa de completitud: 96.97%

3️⃣  TRATAMIENTO DE OUTLIERS
   Valores clipped: 11,958
   Porcentaje de valores modificados: 1.77%

4️⃣  TIPOS DE DATOS
   float64: 33 columnas

5️⃣  INFORMACIÓN TEMPORAL
   Índice temporal: ✅ Configurado
   Rango temporal: 2023-01-01 00:00:00 a 2025-04-30 23:00:00
   Duración total: 850 days 23:00:00

6️⃣  RESUMEN DE COLUMNAS
   Columnas numéricas: 33
   Columnas temporales: 0
   Otras columnas: 0

7️⃣  ESTADÍSTICAS DESCRIPTIVAS (Columnas Numéricas)
   Promedio de medias: 252.59
   Promedio de desv. estándar: 231.57
   Rango de valores mínimos: [-124.46, 82.16]
   Rango de valores máximos: [1.18, 2191.88]

✅ PREPROCESAMIENTO COMPLETADO EXITOSAMENTE


## 5. 💾 Guardado del Dataset Procesado

### 🎯 Objetivo Final
Guardar el dataset completamente procesado en formato CSV para uso en las siguientes fases del proyecto:
- **Feature Engineering** (03_feature_engineering.ipynb)
- **Model Training** (04_model_training.ipynb)
- **Model Evaluation** (05_model_evaluation.ipynb)

In [13]:
def guardar_dataset_final(df, ruta_destino, nombre_base='timeseries_data',
                         archivos_procesados=None, archivos_fallidos=None,
                         valores_interpolados=0, valores_clipped=0):
    """
    Guarda dataset en CSV (siempre) y Parquet (usando subprocess para evitar conflictos)
    Incluye todos los archivos de metadatos
    """
    ruta_destino = Path(ruta_destino)
    ruta_destino.mkdir(parents=True, exist_ok=True)
    
    print(f"💾 Guardando dataset final...")
    print(f"   📊 Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
    
    resultados = {}
    
    # ========== CSV (SIEMPRE) ==========
    try:
        archivo_csv = ruta_destino / f"{nombre_base}.csv"
        print(f"   📄 Guardando CSV...")
        df.to_csv(archivo_csv, index=True, encoding='utf-8')
        tamaño_mb = archivo_csv.stat().st_size / (1024 * 1024)
        print(f"      ✅ CSV: {tamaño_mb:.1f} MB")
        resultados['csv'] = {'exito': True, 'tamaño_mb': tamaño_mb}
    except Exception as e:
        print(f"      ❌ CSV Error: {str(e)}")
        return {'csv': {'exito': False, 'error': str(e)}}
    
    # ========== PARQUET (SUBPROCESS AISLADO) ==========
    archivo_parquet = ruta_destino / f"{nombre_base}.parquet"
    print(f"   📦 Guardando Parquet (subprocess aislado)...")
    
    try:
        # CSV temporal
        temp_csv = ruta_destino / 'temp_for_parquet.csv'
        df.to_csv(temp_csv, index=True)
        
        # Script aislado
        script_content = f'''
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

try:
    df = pd.read_csv("{temp_csv}", index_col=0, parse_dates=True)
    
    # Limpiar para Parquet
    if hasattr(df.index, 'hasnans') and df.index.hasnans:
        df = df[df.index.notna()]
    if str(type(df.index)) == "<class 'pandas.core.indexes.datetimes.DatetimeIndex'>":
        df = df.reset_index()
    
    df.to_parquet("{archivo_parquet}", engine='pyarrow', index=False, compression='snappy')
    print("PARQUET_SUCCESS")
    
except Exception as e:
    print(f"PARQUET_ERROR: {{e}}")
'''
        
        # Ejecutar subprocess
        result = subprocess.run([sys.executable, '-c', script_content], 
                              capture_output=True, text=True, timeout=120)
        
        # Limpiar
        if temp_csv.exists():
            temp_csv.unlink()
        
        # Verificar
        if result.returncode == 0 and "PARQUET_SUCCESS" in result.stdout:
            tamaño_mb = archivo_parquet.stat().st_size / (1024 * 1024)
            print(f"      ✅ Parquet: {tamaño_mb:.1f} MB")
            resultados['parquet'] = {'exito': True, 'tamaño_mb': tamaño_mb}
        else:
            print(f"      ❌ Parquet: Subprocess failed")
            resultados['parquet'] = {'exito': False, 'error': 'subprocess_failed'}
            
    except Exception as e:
        print(f"      ❌ Parquet Exception: {str(e)}")
        resultados['parquet'] = {'exito': False, 'error': str(e)}
    
    # ========== METADATOS COMPLETOS ==========
    try:
        print(f"   📄 Generando archivos de metadatos...")
        
        # 1. Metadatos principales
        archivo_meta = ruta_destino / 'preproceso_metadata.txt'
        with open(archivo_meta, 'w', encoding='utf-8') as f:
            f.write(f"METADATOS DEL PREPROCESAMIENTO\n")
            f.write(f"=" * 40 + "\n\n")
            f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Dimensiones: {df.shape[0]:,} × {df.shape[1]}\n")
            f.write(f"Archivos procesados: {len(archivos_procesados or [])}\n")
            f.write(f"Archivos fallidos: {len(archivos_fallidos or [])}\n")
            f.write(f"Valores interpolados: {valores_interpolados:,}\n")
            f.write(f"Valores clipped: {valores_clipped:,}\n")
            f.write(f"Índice temporal: {'Sí' if hasattr(df.index, 'tz') or 'datetime' in str(type(df.index)).lower() else 'No'}\n")
            f.write(f"\nFormatos guardados:\n")
            for formato, resultado in resultados.items():
                status = "✅" if resultado['exito'] else "❌" 
                f.write(f"  {formato.upper()}: {status}\n")
            
            if archivos_procesados:
                f.write(f"\nArchivos procesados exitosamente:\n")
                for archivo in archivos_procesados:
                    f.write(f"  - {archivo}\n")
            
            if archivos_fallidos:
                f.write(f"\nArchivos que fallaron:\n")
                for archivo in archivos_fallidos:
                    f.write(f"  - {archivo}\n")
        
        # 2. Resumen de columnas
        archivo_columnas = ruta_destino / 'columnas_resumen.csv'
        columnas_info = pd.DataFrame({
            'columna': df.columns,
            'tipo_datos': [str(dtype) for dtype in df.dtypes],
            'valores_no_nulos': df.count(),
            'valores_nulos': df.isnull().sum(),
            'porcentaje_completitud': ((df.count() / len(df)) * 100).round(2)
        })
        
        # Agregar estadísticas para columnas numéricas
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        columnas_info['es_numerica'] = columnas_info['columna'].isin(numeric_cols)
        
        # Estadísticas básicas para columnas numéricas
        columnas_info['min_valor'] = np.nan
        columnas_info['max_valor'] = np.nan
        columnas_info['media'] = np.nan
        
        for col in numeric_cols:
            if not df[col].empty and df[col].notna().any():
                idx = columnas_info['columna'] == col
                try:
                    columnas_info.loc[idx, 'min_valor'] = df[col].min()
                    columnas_info.loc[idx, 'max_valor'] = df[col].max()
                    columnas_info.loc[idx, 'media'] = df[col].mean()
                except:
                    pass  # Ignorar errores en estadísticas
        
        columnas_info.to_csv(archivo_columnas, index=False, encoding='utf-8')
        
        # 3. Estadísticas descriptivas básicas (solo numéricas)
        if len(numeric_cols) > 0:
            archivo_stats = ruta_destino / 'estadistica_basica.csv'
            stats_desc = df[numeric_cols].describe()
            stats_desc.to_csv(archivo_stats, encoding='utf-8')
        
        print(f"      ✅ preproceso_metadata.txt")
        print(f"      ✅ columnas_resumen.csv")
        if len(numeric_cols) > 0:
            print(f"      ✅ estadistica_basica.csv")
        
    except Exception as e:
        print(f"      ⚠️  Error en metadatos: {str(e)}")
        pass
    
    # ========== RESUMEN FINAL ==========
    print(f"\n📋 Resumen del guardado:")
    exitosos = sum(1 for r in resultados.values() if r['exito'])
    for formato, resultado in resultados.items():
        if resultado['exito']:
            print(f"   ✅ {formato.upper()}: {resultado['tamaño_mb']:.1f} MB")
        else:
            print(f"   ❌ {formato.upper()}: Error")
    
    print(f"\n📁 Archivos generados en data/processed/:")
    print(f"   🗃️  {nombre_base}.csv - Dataset principal")
    if resultados.get('parquet', {}).get('exito'):
        print(f"   📦 {nombre_base}.parquet - Dataset comprimido")
    print(f"   📄 preproceso_metadata.txt - Metadatos del procesamiento")
    print(f"   📊 columnas_resumen.csv - Resumen de columnas")
    if len(df.select_dtypes(include=[np.number]).columns) > 0:
        print(f"   📈 estadistica_basica.csv - Estadísticas descriptivas")
    
    print(f"✅ Guardado completado: {exitosos}/{len(resultados)} formatos exitosos")
    
    return resultados
    
exito_guardado = guardar_dataset_final(df_final, ruta_processed)


💾 Guardando dataset final...
   📊 Dimensiones: 20,523 filas × 33 columnas
   📄 Guardando CSV...
      ✅ CSV: 11.8 MB
   📦 Guardando Parquet (subprocess aislado)...
      ✅ Parquet: 3.2 MB
   📄 Generando archivos de metadatos...
      ✅ preproceso_metadata.txt
      ✅ columnas_resumen.csv
      ✅ estadistica_basica.csv

📋 Resumen del guardado:
   ✅ CSV: 11.8 MB
   ✅ PARQUET: 3.2 MB

📁 Archivos generados en data/processed/:
   🗃️  timeseries_data.csv - Dataset principal
   📦 timeseries_data.parquet - Dataset comprimido
   📄 preproceso_metadata.txt - Metadatos del procesamiento
   📊 columnas_resumen.csv - Resumen de columnas
   📈 estadistica_basica.csv - Estadísticas descriptivas
✅ Guardado completado: 2/2 formatos exitosos


## 📈 Resumen del Preprocesamiento

### ✅ Tareas Completadas

1. **Carga y Consolidación Inteligente**
   - ✅ Detección automática de estructura de archivos Excel
   - ✅ Consolidación de 28 archivos en un dataset único
   - ✅ Manejo robusto de diferentes formatos (.xls/.xlsx)

2. **Limpieza de Estructura de Datos**
   - ✅ Normalización de nombres de columnas a formato snake_case
   - ✅ Conversión inteligente de tipos de datos
   - ✅ Manejo de diferentes formatos temporales

3. **Tratamiento de Calidad de Datos**
   - ✅ Interpolación temporal de valores faltantes
   - ✅ Clipping estadístico de outliers (percentiles 1-99)
   - ✅ Preservación de información operacional relevante

4. **Validación y Documentación**
   - ✅ Métricas completas de calidad de datos
   - ✅ Documentación exhaustiva del procesamiento
   - ✅ Archivos de metadatos para trazabilidad

### 🔄 Pipeline de Calidad Implementado

```
Datos Raw → Detección Automática → Limpieza → Conversión de Tipos → 
Interpolación → Clipping → Validación → Dataset Limpio
```

---

**📊 Estado**: ✅ **Preprocesamiento Completado**  
**➡️  Siguiente fase**: `03_feature_engineering.ipynb`